# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tato tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět, jak vypadá její derivace
- celý tento proces lze reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je navenek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funkcí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [32]:
class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()
    
    def diff(self):
        pass
    
    def __add__(self, other):
        return Addition(self, other)
    
    def __sub__(self, other):
        return Addition(self, Multiplication(cst(-1), other))
    
    def __mul__(self, other):
        return Multiplication(self, other)
    
# Base class for binary operations between el. functions
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument
        
# Base class for elementary functions
class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
        
class Addition(BinaryOperator):
    def __str__(self):
        return str(self.levy_argument) + " + " + str(self.pravy_argument)
    
    def diff(self):
        return self.levy_argument.diff() + self.pravy_argument.diff() # Derivative of a sum: (f + g)' = f' + g'
    
class Multiplication(BinaryOperator):
    def __str__(self):
        return "(" + str(self.levy_argument) + ")*(" + str(self.pravy_argument) + ")"
    
    def diff(self):
        return self.levy_argument.diff() * self.pravy_argument + self.levy_argument * self.pravy_argument.diff() # Derivative of a product: (f * g)' = f' * g + f * g'
    
class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return str(self.value)
    
    def diff(self):
        return cst(0) # Derivative of a constant: c' = 0
    
class identity(UnaryOperator):
    def __init__(self):
        pass
    
    def __str__(self):
        return "x"
    
    def diff(self):
        return cst(1) # Derivative of an identical function: x' = 1
    
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent
    
    def __str__(self):
        return "(" + str(self.argument) + ")^" + str(self.exponent)

    def diff(self):
        return cst(self.exponent) * mocnina(self.argument, cst(self.exponent - 1)) * self.argument.diff() # Derivative of a power: (x^n)' = n*(x^(n-1))
    
class sin(UnaryOperator):
    def diff(self):
        return cos(self.argument) * self.argument.diff() # ( sin(f(x)) )' = cos(f(x)) * (f(x))'

class cos(UnaryOperator):
    def diff(self):
        return cst(-1) * sin(self.argument) * self.argument.diff() # ( cos(f(x)) )' = -sin(f(x)) * (f(x))'

class exp(UnaryOperator):
    def diff(self):
        return exp(self.argument) * self.argument.diff()  # ( e^(f(x)) )' = e^(f(x)) * (f(x))'
    
class ln(UnaryOperator):
    def diff(self):
        return frc(self.argument) * self.argument.diff() # ( ln(f(x)) )' = ( 1/(f(x)) ) * (f(x))'

class frc(UnaryOperator):
    def __str__(self):
        return "1/(" + str(self.argument) + ")"
    
    def diff(self):
        return cst(-1) * mocnina(frc(self.argument), 2) * self.argument.diff() # ( 1/(f(x)) )' = ( -1/((f(x))^2) ) * (f(x))'

In [ ]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f1)
print(f1.diff())

můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [ ]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f2)
print(f2.diff())


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [ ]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f3)
print(f3.diff())


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 

In [ ]:
x = identity()
func = frc(mocnina(x, 2))
func2 = mocnina(x, -2)
print(func)
print(func2)
print(func.diff())
print(func2.diff())